<a href="https://colab.research.google.com/github/LemonJam84/rhys-is-a-fish/blob/main/20250820_Upwork_Template_to_JSON_Hand_Data_(Multiple).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
# Combined pipeline: (A) add Win% + Tie% to Tracker, write board strings, then (B) build JSON files.
# Paste this whole cell into Colab and run.

# =========================
# Quick installs (Colab)
# =========================
!pip -q install eval7 openpyxl

# =========================
# CONFIG — EDIT THIS LIST
# =========================
# Put your Excel file names (uploaded to /content) or absolute paths here:
INPUT_FILES = [
    "/content/NYC SNG Demo_Table #3_5-Handed Player Template(@1).xlsx",
    "/content/NYC SNG Demo_Table #3_4-Handed Player Template(@2).xlsx",
]

# Optional: keep the visible (cached) results for ALL formula cells when saving
# (Preserves commas in column F and any other formula-driven text/values.)
PRESERVE_ALL_CACHED_VALUES = True

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 675.3/675.3 kB 26.2 MB/s eta 0:00:00


In [2]:
# =========================
# Imports
# =========================
import os, re, json, random, shutil
from pathlib import Path
from itertools import chain
from datetime import datetime
import pandas as pd
import numpy as np
import eval7
from openpyxl import load_workbook
try:
    from google.colab import files
except Exception:
    class _Files:
        def download(self, *a, **k): pass
    files = _Files()

# =========================
# Helpers
# =========================
def _resolve_inputs(files_list):
    """Resolve names → absolute paths (defaults to /content when given a bare name)."""
    resolved = []
    for f in files_list:
        p = Path(f)
        if not p.is_absolute():
            p = Path("/content") / f
        if not p.exists():
            raise FileNotFoundError(f"Input not found: {p}")
        resolved.append(p)
    return resolved

excel_paths_list = _resolve_inputs(INPUT_FILES)
print("Found workbook(s):")
for p in excel_paths_list:
    print(" -", p)

def freeze_cached_values_across_workbook(file_name: str, wb_edit, sheet_names=None, only_replace_formulas=True):
    """
    Copy cached (displayed) values from the on-disk workbook into the in-memory workbook being edited.
    - file_name: path of the workbook you loaded earlier and are about to save
    - wb_edit:   openpyxl Workbook object you’ve been editing
    - sheet_names: None for all sheets, or a list to limit which sheets are frozen
    - only_replace_formulas: True = only overwrite cells that currently contain formulas ('=...') in wb_edit
    """
    try:
        wb_vals = load_workbook(file_name, data_only=True)  # displays cached results
    except Exception as e:
        print(f"[WARN] Could not open values view to preserve cached data: {e}")
        return

    target_sheets = sheet_names or [ws.title for ws in wb_edit.worksheets]
    for sname in target_sheets:
        if sname not in wb_edit.sheetnames or sname not in wb_vals.sheetnames:
            continue
        ws_edit = wb_edit[sname]
        ws_vals = wb_vals[sname]

        max_row = ws_vals.max_row
        max_col = ws_vals.max_column
        for r in range(1, max_row + 1):
            for c in range(1, max_col + 1):
                cached = ws_vals.cell(row=r, column=c).value
                if cached is None:
                    continue
                if only_replace_formulas:
                    v = ws_edit.cell(row=r, column=c).value
                    if isinstance(v, str) and v.startswith("="):
                        ws_edit.cell(row=r, column=c, value=cached)
                else:
                    ws_edit.cell(row=r, column=c, value=cached)
    try:
        wb_edit.calculation.fullCalcOnLoad = True
    except Exception:
        pass

def col_letter_to_index(letters: str) -> int:
    idx = 0
    for ch in letters:
        idx = idx * 26 + (ord(ch.upper()) - ord('A') + 1)
    return idx - 1

def parse_cards(card_str):
    """Return up to three card image paths for a flop-like string."""
    if not card_str or pd.isna(card_str):
        return ["", "", ""]
    toks = [t.strip().rstrip(",") for t in re.split(r"[,\s]+", str(card_str).strip()) if t.strip()]
    toks = toks[:3]
    out = [f"cards/{t}.png" if t else "" for t in toks]
    while len(out) < 3:
        out.append("")
    return out

# =========================
# Equity engine (eval7)
# =========================
def equity_eval7(hands, iters=100_000, board=None, folded=None, print_time=False):
    """Returns two dicts: win% and tie% for each hand string in `hands`."""
    start = datetime.now()

    def try_card(card_str):
        try:
            return eval7.Card(card_str.strip())
        except:
            return None

    # parse hands
    player_hands = [
        [c for c in (try_card(tok) for tok in str(hand).replace(",", " ").split()) if c]
        for hand in hands
    ]
    used_cards = list(chain.from_iterable(player_hands))

    # parse board
    if board:
        tokens = board if isinstance(board, list) else re.split(r"[,\s]+", str(board))
        board_cards = [c for c in (try_card(tok) for tok in tokens) if c]
    else:
        board_cards = []
    used_cards += board_cards

    # deck
    deck = [c for c in eval7.Deck() if c not in used_cards]
    n = len(player_hands)

    # init
    win_count = [0]*n
    tie_count = [0]*n

    # folded mask
    if folded is None:
        folded = [False]*n
    if len(folded) != n:
        raise ValueError("Length of 'folded' must match number of hands")
    live = [i for i,f in enumerate(folded) if not f]
    if not live:
        return ({h:0.0 for h in hands}, {h:0.0 for h in hands})

    # deterministic if full board
    if len(board_cards) == 5:
        scores = [None]*n
        for i in live:
            scores[i] = eval7.evaluate(player_hands[i] + board_cards)
        best = max(s for s in scores if s is not None)
        winners = [i for i,s in enumerate(scores) if s == best]
        if len(winners) == 1:
            win_count[winners[0]] = 1
        else:
            for w in winners:
                tie_count[w] = 1
        total = 1
    else:
        for _ in range(iters):
            draw = random.sample(deck, 5 - len(board_cards))
            board_full = board_cards + draw
            scores = [None]*n
            for i in live:
                scores[i] = eval7.evaluate(player_hands[i] + board_full)
            best = max(s for s in scores if s is not None)
            winners = [i for i,s in enumerate(scores) if s == best]
            if len(winners) == 1:
                win_count[winners[0]] += 1
            else:
                for w in winners:
                    tie_count[w] += 1
        total = iters

    win_pct = {hands[i]: round(win_count[i] / total, 3) for i in range(n)}
    tie_pct = {hands[i]: round(tie_count[i] / total, 3) for i in range(n)}

    if print_time:
        print("Elapsed:", (datetime.now() - start).total_seconds(), "s")

    return win_pct, tie_pct

# This function reads globals `positions` and `poker_streets` (set inside the writer below)
def equity_eval_calculator_by_row(row):
    hands = [row[pos] for pos in positions]

    streets_to_compute = [
        st for st in poker_streets
        if st == "Pre" or not pd.isna(row[f"{st} {'Cards' if st == 'Flop' else 'Card'}"])
    ]

    board_map = {}
    folded_street_map = {}
    equity_evaluation = {}

    for st in streets_to_compute:
        if st == "Pre":
            board_map[st] = None
            folded_street_map[st] = [False] * len(hands)
        elif st == "Flop":
            flop_tokens = [t for t in re.split(r"[,\s]+", str(row["Flop Cards"]).strip()) if t]
            board_map[st] = flop_tokens[:3]
            folded_street_map[st] = [row.get(f"{pos}_live_{st}", "N") != "Y" for pos in positions]
        elif st == "Turn":
            board_map[st] = board_map["Flop"] + [str(row["Turn Card"]).strip()]
            folded_street_map[st] = [row.get(f"{pos}_live_{st}", "N") != "Y" for pos in positions]
        else:  # River
            board_map[st] = board_map["Turn"] + [str(row["River Card"]).strip()]
            folded_street_map[st] = [row.get(f"{pos}_live_{st}", "N") != "Y" for pos in positions]

        if not all(folded_street_map[st]):
            wins, ties = equity_eval7(
                hands,
                iters=100_000,
                board=board_map[st],
                folded=folded_street_map[st]
            )
        else:
            wins = ties = {h: 0.0 for h in hands}

        equity_evaluation[f"{st}_Win"] = wins
        equity_evaluation[f"{st}_Tie"] = ties

    return equity_evaluation

# ============================================
# SCRIPT A — Write equities + board to Tracker
# ============================================
def run_equity_writeback_on_file(file_name: str):
    """
    Writes EN–EZ (win/tie by street/seat) and copies board strings from the
    ORIGINAL uploaded 'Poker Session Tracker' sheet (pandas read) back into:
      - U  (col 21): Flop string
      - AJ (col 36): Turn card
      - AY (col 51): River card
    on the first player's row of each hand block.
    """
    print(f"\n[Equities] Processing {file_name}…")
    directory = os.path.dirname(file_name) or "/content"
    os.chdir(directory)

    # 1) Player Hands (for seats, cards, etc.—unchanged)
    player_hands_table = pd.read_excel(file_name, sheet_name='Player Hands', header=[2])
    player_hands_table = player_hands_table.iloc[:, 1:]   # drop index-like column if present
    blank_cols = [col for col in player_hands_table.columns
                  if str(col).startswith("Unnamed") or str(col).strip() == ""]
    if blank_cols:
        player_hands_table.rename(columns={blank_cols[0]: "Output Quality"}, inplace=True)
    player_hands_table = player_hands_table.dropna(subset={"Small Blind"})
    number_of_players = list(player_hands_table.columns).index("Output Quality") - 1
    number_of_hands   = len(player_hands_table)

    # 2) Poker Session Tracker (ORIGINAL values snapshot via pandas)
    #    -> This is the source of truth for flop/turn/river strings.
    start_header_row_on_tracker_sheet = 17  # header row used when reading via pandas
    poker_tracker_sheet = pd.read_excel(
        file_name, sheet_name='Poker Session Tracker', header=[start_header_row_on_tracker_sheet]
    )

    # Column layout (0-based for the pandas DataFrame we just read)
    start_row = 0
    distance_between_hand_numbers = 17
    player_position_column = 3   # D
    player_name_column     = 4   # E
    flop_player_live_column  = 19
    turn_player_live_column  = 34
    river_player_live_column = 49

    # >>> IMPORTANT: board string columns on the ORIGINAL Tracker (pandas indices)
    FLOP_STR_COL  = 20  # Excel U
    TURN_STR_COL  = 35  # Excel AJ
    RIVER_STR_COL = 50  # Excel AY

    # 3) Build per-hand player dict (positions + live flags) from ORIGINAL Tracker
    data_by_hand_dict = {}
    for hand in range(1, number_of_hands + 1):
        hand_start = distance_between_hand_numbers * (hand - 1) + start_row
        hand_data_by_player = {}
        for i in range(number_of_players):
            row_idx = hand_start + i
            player_position = poker_tracker_sheet.iat[row_idx, player_position_column]
            hand_data_by_player[player_position] = {
                "name":       poker_tracker_sheet.iat[row_idx, player_name_column],
                "live Flop":  poker_tracker_sheet.iat[row_idx, flop_player_live_column],
                "live Turn":  poker_tracker_sheet.iat[row_idx, turn_player_live_column],
                "live River": poker_tracker_sheet.iat[row_idx, river_player_live_column],
            }
        data_by_hand_dict[hand] = hand_data_by_player

    player_hands_table["Data for the Hand"] = player_hands_table["Hand Number"].apply(
        lambda x: data_by_hand_dict[x]
    )

    # Flatten to columns (e.g., "Seat 1_live_Flop")
    def flatten_hand_data(hand_data):
        flat = {}
        for position, info in hand_data.items():
            for attr_key, attr_val in info.items():
                flat[f"{position}_{attr_key.replace(' ', '_')}"] = attr_val
        return flat

    flat_df = player_hands_table["Data for the Hand"].apply(flatten_hand_data).apply(pd.Series)
    player_hands_table = pd.concat([player_hands_table, flat_df], axis=1)

    # 4) Equities (unchanged)
    global positions, poker_streets
    positions = list(player_hands_table.columns)[1:number_of_players+1]
    poker_streets = ["Pre", "Flop", "Turn", "River"]

    player_hands_table["Equity Eval"] = player_hands_table.apply(equity_eval_calculator_by_row, axis=1)

    def get_tie_val(row, street, pos):
        return row["Equity Eval"].get(f"{street}_Tie", {}).get(row[pos], 0.0)

    for street in poker_streets:
        for pos in positions:
            player_hands_table[f"{street}_Win_{pos}"] = player_hands_table.apply(
                lambda row, s=street, p=pos: row["Equity Eval"].get(f"{s}_Win", {}).get(row[p], 0.0), axis=1
            )
            player_hands_table[f"{street}_Tie_{pos}"] = player_hands_table.apply(
                get_tie_val, axis=1, args=(street, pos)
            )

    player_hands_table.drop(columns=["Equity Eval"], inplace=True)

    # 5) Write back to 'Poker Session Tracker' (wins/ties + board strings)
    from openpyxl import load_workbook
    wb = load_workbook(file_name)
    ws = wb["Poker Session Tracker"]

    # Excel is 1-based rows; first player row is header(17) + 2 => row 19
    first_data_row = start_header_row_on_tracker_sheet + 2  # 19

    # Where to write win/tie values
    win_cols = {"Pre": "EN", "Flop": "EO", "Turn": "EP", "River": "EQ"}
    tie_cols = {"Pre": "EW", "Flop": "EX", "Turn": "EY", "River": "EZ"}

    for hand in range(1, number_of_hands + 1):
        block_start = distance_between_hand_numbers * (hand - 1)

        # ---- a) write Win/Tie per seat (unchanged) ----
        df_row = player_hands_table.loc[player_hands_table["Hand Number"] == hand].iloc[0]
        for i in range(number_of_players):
            row_idx   = block_start + i
            excel_row = first_data_row + row_idx
            pos = ws.cell(row=excel_row, column=player_position_column + 1).value  # read position label
            for street in poker_streets:
                ws[f"{win_cols[street]}{excel_row}"] = df_row[f"{street}_Win_{pos}"]
                ws[f"{tie_cols[street]}{excel_row}"] = df_row[f"{street}_Tie_{pos}"]

        # ---- b) board strings from ORIGINAL Tracker (pandas), first player row only ----
        pandas_first_row = block_start  # because pandas data rows start at Excel row 19 as index 0
        orig_flop  = poker_tracker_sheet.iat[pandas_first_row, FLOP_STR_COL]
        orig_turn  = poker_tracker_sheet.iat[pandas_first_row, TURN_STR_COL]
        orig_river = poker_tracker_sheet.iat[pandas_first_row, RIVER_STR_COL]

        flop_str  = "" if pd.isna(orig_flop)  else str(orig_flop).strip()
        turn_str  = "" if pd.isna(orig_turn)  else str(orig_turn).strip()
        river_str = "" if pd.isna(orig_river) else str(orig_river).strip()

        excel_row_first_player = first_data_row + block_start
        ws.cell(row=excel_row_first_player, column=21).value = flop_str   # U
        ws.cell(row=excel_row_first_player, column=36).value = turn_str   # AJ
        ws.cell(row=excel_row_first_player, column=51).value = river_str  # AY

    # 6) Preserve cached values (optional)
    if globals().get("PRESERVE_ALL_CACHED_VALUES", False):
        try:
            freeze_cached_values_across_workbook(
                file_name=file_name,
                wb_edit=wb,
                sheet_names=None,
                only_replace_formulas=True
            )
        except Exception as e:
            print("[WARN] Skipping cache freeze:", e)

    wb.save(file_name)
    try:
        files.download(file_name)
    except Exception:
        pass

    return number_of_players

# ============================================
# SCRIPT B — Build JSON from updated workbook
# ============================================
def get_num_players_from_path(path: Path) -> int:
    """
    Extracts the integer immediately before '-H' in the filename stem.
    E.g. '..._5-Handed ...' → 5
    """
    stem = path.stem
    m = re.search(r'(\d+)(?=-H)', stem)
    if not m:
        raise ValueError(f"Could not find a number before '-H' in {stem!r}")
    return int(m.group(1))

def extract_actions(row, col_range):
    acts = []
    for i in range(0, len(col_range), 2):
        action = str(row[col_range[i]]).strip().upper() if not pd.isna(row[col_range[i]]) else ""
        size   = row[col_range[i+1]] if i+1 < len(col_range) and not pd.isna(row[col_range[i+1]]) else ""
        if action in {"X", "PROCEED TO NEXT STAGE", "PENDING", ""}:
            continue
        acts.append({
            "type": action,
            "size": float(size) if str(size).replace('.', '', 1).isdigit() else 0.0
        })
    return acts

def build_json_from_updated_workbook(excel_path: Path):
    print(f"\n[JSON] Processing {excel_path.name}…")

    sheet_name = "Poker Session Tracker"
    NUM_PLAYERS = get_num_players_from_path(excel_path)

    # 1) load sheet (as raw values)
    df_tracker = pd.read_excel(excel_path, sheet_name=sheet_name, header=None, dtype=str)
    col_f      = df_tracker.iloc[:, 5].fillna("")
    has_comma  = col_f.str.contains(",", regex=False)
    if not has_comma.any():
        raise ValueError("No comma found in column F of the tracker sheet!")
    last_idx = has_comma[has_comma].index[-1]
    last_row = last_idx + 1

    # 2) read again (numeric+string mix), no header
    df = pd.read_excel(excel_path, sheet_name=sheet_name, header=None)

    # first player row is 19 (Excel), then every 17 rows; block spans NUM_PLAYERS rows
    hand_row_ranges = [(r - 1, r - 1 + NUM_PLAYERS - 1) for r in range(19, last_row, 17)]

    stage_names = ["preflop", "flop", "turn", "river"]
    stage_columns = {
        "preflop": list(range(6, 18)),
        "flop":    list(range(21, 33)),
        "turn":    list(range(36, 48)),
        "river":   list(range(51, 63)),
    }

    pot_columns = {
        "main":  ["DJ", "DK", "DL", "DM"],
        "side1": ["DO", "DP", "DQ", "DR"],
        "side2": ["DT", "DU", "DV", "DW"],
        "side3": ["DY", "DZ", "EA", "EB"],
        "side4": ["ED", "EE", "EF", "EG"],
        "side5": ["EI", "EJ", "EK", "EL"],
    }

    equity_cols = {"preflop": "EN", "flop": "EO", "turn": "EP", "river": "EQ"}
    tie_cols    = {"preflop": "EW", "flop": "EX", "turn": "EY", "river": "EZ"}

    equity_idx = {st: col_letter_to_index(col) for st, col in equity_cols.items()}
    tie_idx    = {st: col_letter_to_index(col) for st, col in tie_cols.items()}

    def get_pot_ends(row_offset):
        pots = {st: {p: 0 for p in pot_columns} for st in stage_names}
        for ptype, cols in pot_columns.items():
            for i, st in enumerate(stage_names):
                col_idx = col_letter_to_index(cols[i])
                val     = df.iloc[row_offset, col_idx]
                pots[st][ptype] = float(val) if pd.notna(val) else 0
        return pots

    def get_fold_stage(acts_by_stage):
        result = {}
        for i in range(NUM_PLAYERS):
            result[i] = None
            for st in stage_names:
                if any(a["type"] == "F" for a in acts_by_stage[i][st]):
                    result[i] = st
                    break
        return result

    # Prepare output folder: /content/Hand_Data/<workbook_stem>
    output_root = Path("/content/Hand_Data")
    output_dir  = output_root / excel_path.stem
    if output_dir.exists():
        shutil.rmtree(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    output_files = []

    for hand_no, (start, end) in enumerate(hand_row_ranges, 1):
        # community cards from first row of block (U/AJ/AY)
        community = {
            "flop":  str(df.iloc[start, 20]) if not pd.isna(df.iloc[start, 20]) else "",
            "turn":  str(df.iloc[start, 35]) if not pd.isna(df.iloc[start, 35]) else "",
            "river": str(df.iloc[start, 50]) if not pd.isna(df.iloc[start, 50]) else "",
        }

        # players (name, chips, cards + equities)
        players = []
        equity_by_player = {i: {} for i in range(NUM_PLAYERS)}
        for i in range(NUM_PLAYERS):
            row   = df.iloc[start + i]
            name  = row[4] if not pd.isna(row[4]) else f"Player{i+1}"
            chips = float(row[2]) if not pd.isna(row[2]) else 0.0
            cards = [c.strip().rstrip(',') for c in str(row[5]).split()] if not pd.isna(row[5]) else ["",""]

            for st in stage_names:
                eq_val = row[equity_idx[st]]
                equity_by_player[i][st] = float(eq_val) if pd.notna(eq_val) else 0.0

            players.append({
                "name":  name,
                "chips": chips,
                "card1": f"cards/{cards[0]}.png" if len(cards) > 0 else "",
                "card2": f"cards/{cards[1]}.png" if len(cards) > 1 else "",
            })

        # tie %
        tie_by_player = {i: {} for i in range(NUM_PLAYERS)}
        for i in range(NUM_PLAYERS):
            row = df.iloc[start + i]
            for st in stage_names:
                raw = row[tie_idx[st]]
                tie_by_player[i][st] = float(raw) if pd.notna(raw) else 0.0

        # actions
        acts_by_stage = {
            i: {st: extract_actions(df.iloc[start + i], stage_columns[st]) for st in stage_names}
            for i in range(NUM_PLAYERS)
        }
        fold_stage   = get_fold_stage(acts_by_stage)
        pot_end_vals = get_pot_ends(start)

        # Build per-stage pot start/end
        pot_values = {}
        for idx, st in enumerate(stage_names):
            pot_values[st] = {
                p: {
                    "start": 0 if idx == 0 else pot_end_vals[stage_names[idx-1]][p],
                    "end":   pot_end_vals[st][p],
                }
                for p in pot_columns
            }

        # assemble game_data
        game_data = {}
        for st in stage_names:
            game_data[st] = {
                "players": [],
                "communityCards": {"flop1": "", "flop2": "", "flop3": "", "turn": "", "river": ""},
                "pots": pot_values[st],
            }

            # community cards
            if st in {"flop", "turn", "river"}:
                f1, f2, f3 = parse_cards(community["flop"])
                game_data[st]["communityCards"].update({"flop1": f1, "flop2": f2, "flop3": f3})
            if st in {"turn", "river"}:
                t = community["turn"].strip().rstrip(',') if community["turn"] else ""
                game_data[st]["communityCards"]["turn"] = f"cards/{t}.png" if t else ""
            if st == "river":
                r = community["river"].strip().rstrip(',') if community["river"] else ""
                game_data[st]["communityCards"]["river"] = f"cards/{r}.png" if r else ""

            # player objects per stage (respect folding)
            for i in range(NUM_PLAYERS):
                base     = players[i]
                st_idx   = stage_names.index(st)
                fold_idx = stage_names.index(fold_stage[i]) if fold_stage[i] else None
                folded   = fold_idx is not None and fold_idx < st_idx

                game_data[st]["players"].append({
                    "name":   base["name"],
                    "chips":  base["chips"],
                    "card1":  base["card1"] if not folded else "cards/grey_back.jpg",
                    "card2":  base["card2"] if not folded else "cards/grey_back.jpg",
                    "equity": equity_by_player[i][st],
                    "tie":    tie_by_player[i][st],
                    "actions": acts_by_stage[i][st],
                })

        # write file
        out_file = (Path("/content/Hand_Data") / excel_path.stem) / f"gameData_hand{hand_no}.js"
        out_file.write_text(f"const gameData = {json.dumps(game_data, indent=2)};\n")
        print("   wrote:", out_file)
        output_files.append(out_file)

    # zip all JSONs for this workbook
    zip_no_ext = f"/content/{excel_path.stem}_Hand_Data"
    shutil.make_archive(zip_no_ext, "zip", Path("/content/Hand_Data") / excel_path.stem)
    zip_path = f"{zip_no_ext}.zip"
    print("   packaged:", zip_path)
    try:
        files.download(zip_path)
    except Exception:
        pass

# =========================
# Orchestration: A then B
# =========================
for src_path in excel_paths_list:
    # A) Write equities into THIS workbook (in place)
    _ = run_equity_writeback_on_file(str(src_path))

    # B) Build JSON from the updated workbook (original JSON logic)
    build_json_from_updated_workbook(src_path)

print("\n🎉 Finished all workbooks.")


Found workbook(s):
 - /content/NYC SNG Demo_Table #3_5-Handed Player Template(@1).xlsx
 - /content/NYC SNG Demo_Table #3_4-Handed Player Template(@2).xlsx

[Equities] Processing /content/NYC SNG Demo_Table #3_5-Handed Player Template(@1).xlsx…


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


[JSON] Processing NYC SNG Demo_Table #3_5-Handed Player Template(@1).xlsx…
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand1.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand2.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand3.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand4.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand5.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand6.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand7.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand8.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_5-Handed Player Template(@1)/gameData_hand9.js
   wrote: /content/Hand_Data/NYC SNG Demo_

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>


[JSON] Processing NYC SNG Demo_Table #3_4-Handed Player Template(@2).xlsx…
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_4-Handed Player Template(@2)/gameData_hand1.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_4-Handed Player Template(@2)/gameData_hand2.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_4-Handed Player Template(@2)/gameData_hand3.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_4-Handed Player Template(@2)/gameData_hand4.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_4-Handed Player Template(@2)/gameData_hand5.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_4-Handed Player Template(@2)/gameData_hand6.js
   wrote: /content/Hand_Data/NYC SNG Demo_Table #3_4-Handed Player Template(@2)/gameData_hand7.js
   packaged: /content/NYC SNG Demo_Table #3_4-Handed Player Template(@2)_Hand_Data.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 Finished all workbooks.
